## Imports and global variables

In [22]:
import numpy as np

import sys
sys.path.append("./src/")

In [23]:
from implementations import *
from helpers import *

%load_ext autoreload
%autoreload 2
%matplotlib inline

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [24]:
DATA_FOLDER = './data/'
TRAIN_FILE = './train.csv'
TEST_FILE = './train.csv'

NAN_VALUE = -999.0

# Import dataset
Import dataset and apply basic transformations to be able to use it afterwards : 
- Replace labels chars by integers
- Standardize training df
- Replace missing values by the mean

In [25]:
# Load data
x, y = load_data(DATA_FOLDER + TRAIN_FILE, sub_sample=True)

# Standardize before replacing missing values
x, _, _ = standardize(x, missing_values=True)

# Replace missing data by the mean
x[x == NAN_VALUE] = np.nan
x = replace_nan_by_means(x)

assert(x[np.isnan(x)].shape[0] == 0)
x

array([[ 0.73640279,  0.53680972,  0.64296201, ...,  0.42090251,
         0.41236149,  0.67898831],
       [ 0.78805581,  0.57615357,  0.65539534, ..., -1.87871129,
        -1.87871129,  0.52432811],
       [-1.87871129,  0.79089515,  0.70762543, ..., -1.87871129,
        -1.87871129,  0.51978746],
       ...,
       [ 0.71711136,  0.54067214,  0.64206078, ..., -1.87871129,
        -1.87871129,  0.41805167],
       [ 0.80322273,  0.51295005,  0.70154441, ..., -1.87871129,
        -1.87871129,  0.41805167],
       [ 0.63739828,  0.46767462,  0.57384991, ..., -1.87871129,
        -1.87871129,  0.41805167]])

In [26]:
# Add bias term to x
xt = np.hstack((np.ones(x.shape[0])[:, np.newaxis], x))

# Split train and test sets
x_tr, x_te, y_tr, y_te = split_data(xt, y, 0.2)

# Fit a model
w, loss_tr = least_squares(y_tr, x_tr)
loss_te = compute_mse(y_te, x_te, w)

print("Training loss : %f" % loss_tr)
print("Test loss : %f" % loss_te)

Training loss : 0.000000
Test loss : 15.623270
